In [1]:
import os
import sys
sys.executable


'C:\\Users\\aa\\myy\\Scripts\\python.exe'

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
print(sys.version)

3.7.6 (tags/v3.7.6:43364a7ae0, Dec 19 2019, 00:42:30) [MSC v.1916 64 bit (AMD64)]


In [3]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [10]:
from hypers import *
from datetime import datetime
import time
import random
import numpy as np
import os
from nltk.tokenize import word_tokenize
import json

def trans2tsp(timestr):
    return int(time.mktime(datetime.strptime(timestr, '%m/%d/%Y %I:%M:%S %p').timetuple()))

def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

def shuffle(pn,labeler,pos):
    index=np.arange(pn.shape[0])
    pn=pn[index]
    labeler=labeler[index]
    pos=pos[index]
    
    for i in range(pn.shape[0]):
        index=np.arange(npratio+1)
        pn[i,:]=pn[i,index]
        labeler[i,:]=labeler[i,index]
    return pn,labeler,pos

def read_news(path,filenames):
    news={}
    category=[]
    subcategory=[]
    news_index={}
    index=1
    word_dict={}
    word_index=1
    with open(os.path.join(path,filenames), encoding='utf-8') as f:
        lines=f.readlines()
    for line in lines:
        splited = line.strip('\n').split('\t')
        doc_id,vert,subvert,title= splited[0:4]
        news_index[doc_id]=index
        index+=1
        category.append(vert)
        subcategory.append(subvert)
        title = title.lower()
        title=word_tokenize(title)
        news[doc_id]=[vert,subvert,title]
        for word in title:
            word = word.lower()
            if not(word in word_dict):
                word_dict[word]=word_index
                word_index+=1
    category=list(set(category))
    subcategory=list(set(subcategory))
    category_dict={}
    index=1
    for c in category:
        category_dict[c]=index
        index+=1
    subcategory_dict={}
    index=1
    for c in subcategory:
        subcategory_dict[c]=index
        index+=1
    return news,news_index,category_dict,subcategory_dict,word_dict

def get_doc_input(news,news_index,category,subcategory,word_dict):
    news_num=len(news)+1
    news_title=np.zeros((news_num,MAX_SENTENCE),dtype='int32')
    news_vert=np.zeros((news_num,),dtype='int32')
    news_subvert=np.zeros((news_num,),dtype='int32')
    for key in news:    
        vert,subvert,title=news[key]
        doc_index=news_index[key]
        news_vert[doc_index]=category[vert]
        news_subvert[doc_index]=subcategory[subvert]
        for word_id in range(min(MAX_SENTENCE,len(title))):
            news_title[doc_index,word_id]=word_dict[title[word_id].lower()]
    return news_title,news_vert,news_subvert

def load_matrix(embedding_path,word_dict):
    embedding_matrix = np.zeros((len(word_dict)+1,300))
    have_word=[]
    with open(os.path.join(embedding_path,'glove.840B.300d.txt'),'rb') as f:
        while True:
            l=f.readline()
            if len(l)==0:
                break
            l=l.split()
            word = l[0].decode()
            if word in word_dict:
                index = word_dict[word]
                tp = [float(x) for x in l[1:]]
                embedding_matrix[index]=np.array(tp)
                have_word.append(word)
    return embedding_matrix,have_word

def read_clickhistory_with_click_counts(news_index, data_root_path, filename):
    with open(os.path.join(data_root_path, filename)) as f:
        lines = f.readlines()

    sessions = []
    global_click_counts = {}  # Dictionary to store global click counts

    for line in lines:
        _, uid, eventime, click, imps = line.strip().split('\t')

        if click == '':
            clicks = []
        else:
            clicks = click.split()

        true_click = []
        for click in clicks:
            if click in news_index:
                true_click.append(click)
                global_click_counts[click] = global_click_counts.get(click, 0) + 1  # Counting clicks

        pos = []
        neg = []
        for imp in imps.split():
            docid, label = imp.split('-')
            if label == '1':
                pos.append(docid)
            else:
                neg.append(docid)

        sessions.append([true_click, pos, neg])

    return sessions, global_click_counts


def parse_user(news_index,session):
    user_num = len(session)
    user={'click': np.zeros((user_num,MAX_ALL),dtype='int32'),}
    for user_id in range(len(session)):
        tclick = []
        click, pos, neg =session[user_id]
        for i in range(len(click)):
            tclick.append(news_index[click[i]])
        click = tclick

        if len(click) >MAX_ALL:
            click = click[-MAX_ALL:]
        else:
            click=[0]*(MAX_ALL-len(click)) + click
            
        user['click'][user_id] = np.array(click)
    return user

def get_train_input(news_index,session,global_click_counts):
    sess_pos = []
    sess_neg = []
    user_id = []
    sorted_news_by_clicks = sorted(global_click_counts, key=global_click_counts.get, reverse=True)

    for sess_id in range(len(session)):
        sess = session[sess_id]
        _, poss, negs=sess
        for i in range(len(poss)):
            pos = poss[i]
            neg = newsample(sorted_news_by_clicks, npratio)
            sess_pos.append(pos)
            sess_neg.append(neg)
            user_id.append(sess_id)
    sess_all = np.zeros((len(sess_pos),1+npratio),dtype='int32')
    label = np.zeros((len(sess_pos),1+npratio))
    for sess_id in range(sess_all.shape[0]):
        pos = sess_pos[sess_id]
        negs = sess_neg[sess_id]
        sess_all[sess_id,0] = news_index[pos]
        index = 1
        for neg in negs:
            sess_all[sess_id,index] = news_index[neg]
            index+=1
        label[sess_id,0]=1
    user_id = np.array(user_id, dtype='int32')
    
    return sess_all, user_id, label

def get_test_input(news_index,session):
    
    Impressions = []
    userid = []
    for sess_id in range(len(session)):
        _, poss, negs = session[sess_id]
        imp = {'labels':[],
                'docs':[]}
        userid.append(sess_id)
        for i in range(len(poss)):
            docid = news_index[poss[i]]
            imp['docs'].append(docid)
            imp['labels'].append(1)
        for i in range(len(negs)):
            docid = news_index[negs[i]]
            imp['docs'].append(docid)
            imp['labels'].append(0)
        Impressions.append(imp)
        
    userid = np.array(userid,dtype='int32')
    
    return Impressions, userid,

def load_news_entity(news_index,KG_root_path):
    with open(os.path.join(KG_root_path,'Release_Small_title.tsv')) as f:
        lines = f.readlines()
    
    EntityId2Index = {}
    ctt = 1
    
    news_entity = {}
    g = []
    for i in range(len(lines)):
        d = json.loads(lines[i].strip('\n'))
        docid = d['doc_id']
        if not docid in news_index:
            continue
        news_entity[docid] = []
        entities = d['entities']
        for j in range(len(entities)):
            e = entities[j]['Label']
            eid = entities[j]['WikidataId']
            if not eid in EntityId2Index:
                EntityId2Index[eid] = ctt
                ctt += 1
            news_entity[docid].append([e,eid,EntityId2Index[eid]])
    
    meta_news_entity = {}
    news_entity2 = {}
    
    
    news_entity_id = {}
    for nid in news_entity:
        news_entity_id[nid] = []
        for e in news_entity[nid]:
            news_entity_id[nid].append(e[-2])
        news_entity_id[nid] = set(news_entity_id[nid])
        
    
    for docid in news_entity:
        meta_news_entity[docid] = news_entity[docid]
        news_entity2[docid] = []
        for v in news_entity[docid]:
            news_entity2[docid].append(v[-1])
        news_entity2[docid] = list(set(news_entity2[docid]))[:5]
        news_entity2[docid] = news_entity2[docid] + [0]*(5-len(news_entity2[docid]))
        news_entity2[docid] = np.array(news_entity2[docid])
    
    news_entity_np = np.zeros((len(news_entity2)+1,5),dtype='int32')
    for nid in news_index:
        nix = news_index[nid]
        news_entity_np[nix] = news_entity2[nid]
        
    return news_entity_id,news_entity_np,EntityId2Index

def load_entity_embedding(KG_root_path,EntityId2Index):
    entity_emb = np.zeros((len(EntityId2Index)+1,100))
    import pickle
    with open(os.path.join(KG_root_path,'title_entity_emb.pkl'),'rb') as f:
        title_entity_emb = pickle.load(f)
    for eid in EntityId2Index:
        eix = EntityId2Index[eid]
        entity_emb[eix] = title_entity_emb[eid]
    return entity_emb

In [11]:
# from preprocessing import *
from models import *
from hypers import *
from utils import *

In [12]:
# import site
# import sys

# def set_default_encoding():
#     sys.setdefaultencoding('utf-8')

# site.setquit(set_default_encoding)
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
data_root_path = 'D:\mind\out'
embedding_path = 'D:\embedding_path'
KG_root_path = 'D:\Hirec\HieRec_KGData'

In [14]:
news,news_index,category_dict,subcategory_dict,word_dict = read_news(data_root_path,'docs.tsv')
# Load data

news_title,news_vert,news_subvert=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict)
news_entity,news_entity_np,EntityId2Index = load_news_entity(news_index,KG_root_path)
news_info = np.concatenate([news_title,news_entity_np],axis=-1)

In [15]:
sessions, global_click_counts = read_clickhistory_with_click_counts(news_index, data_root_path, 'train.tsv')

# Sort the articles based on click counts in descending order
sorted_click_counts = sorted(global_click_counts.items(), key=lambda x: x[1], reverse=True)

# This gives you a list of tuples where the first element is the news article ID and the second is the click count

train_session = read_clickhistory(news_index,data_root_path,'train.tsv')
train_user = parse_user(news_index,train_session)
# Load data
# Use the modified get_train_input function
train_sess, train_user_id, train_label = get_train_input(news_index, sessions, global_click_counts)

# train_sess, train_user_id, train_label = get_train_input(news_index,train_session)

In [16]:
test_session = read_clickhistory(news_index,data_root_path,'test.tsv')
test_user = parse_user(news_index,test_session)
test_impressions, test_userids = get_test_input(news_index,test_session)

In [17]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)
entity_emb_matrix = load_entity_embedding(KG_root_path,EntityId2Index)

In [18]:
index2nid = {}
for nid, nix in news_index.items():
    index2nid[nix] = nid

In [19]:
vert_subvert_mask_table = np.zeros((1,len(category_dict),len(subcategory_dict)))
for nid in range(1,len(news_vert)):
    v = news_vert[nid]-1
    sv = news_subvert[nid]-1
    vert_subvert_mask_table[0,v,sv] = 1

In [20]:
from keras.utils import Sequence

class get_hir_train_generator(Sequence):
    def __init__(self,mask_prob,news_scoring,index2nid,news_vert, subvert,news_entity, news_entity_id, clicked_news,user_id, news_id, label, batch_size):
        self.news_emb = news_scoring
        self.vert = news_vert
        self.subvert = subvert
        self.entity = news_entity
        self.entity_id = news_entity_id
        self.index2nid = index2nid
        
        self.clicked_news = clicked_news

        self.user_id = user_id
        self.doc_id = news_id
        self.label = label
        
        self.mask_prob = mask_prob
        
        self.batch_size = batch_size
        self.ImpNum = self.label.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))
    
    def __get_news(self,docids):
        news_emb = self.news_emb[docids]
        vert = self.vert[docids]
        subvert = self.subvert[docids]
        entity = self.entity[docids]
        return news_emb, vert, subvert, entity
        

    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
            
        label = self.label[start:ed]

        doc_ids = self.doc_id[start:ed]
        title, vert, subvert, entity = self.__get_news(doc_ids)
        
        user_ids = self.user_id[start:ed]
        clicked_ids = self.clicked_news[user_ids]
        user_title, user_vert, user_subvert, user_entity = self.__get_news(clicked_ids)
        
        vert_subvert_mask_input = np.zeros((len(user_subvert),len(category_dict),len(subcategory_dict),))
        for bid in range(len(user_subvert)):
            for nid in range(len(user_subvert[bid])):
                sv = user_subvert[bid][nid]
                if sv ==0:
                    continue
                sv -= 1
                vert_subvert_mask_input[bid,:,sv] = 1
        vert_subvert_mask_input = vert_subvert_mask_input*vert_subvert_mask_table

        
        
        user_vert = keras.utils.to_categorical(user_vert,len(category_dict)+1)
        user_vert = user_vert.transpose((0,2,1))
        user_vert = user_vert[:,1:,:]
        user_vert_mask = user_vert.sum(axis=-1)
        
        vert = keras.utils.to_categorical(vert,len(category_dict)+1)
        vert = vert[:,:,1:]
        
        user_subvert = keras.utils.to_categorical(user_subvert,len(subcategory_dict)+1)
        user_subvert = user_subvert.transpose((0,2,1))
        user_subvert = user_subvert[:,1:,:]
        user_subvert_mask = user_subvert.sum(axis=-1)
                
        subvert = keras.utils.to_categorical(subvert,len(subcategory_dict)+1)
        subvert = subvert[:,:,1:]
    
        user_vert_num = np.array(user_vert.sum(axis=-1),dtype='int32')
        user_subvert_num = np.array(user_subvert.sum(axis=-1),dtype='int32')

        user_subvert_mask = np.array(user_subvert_mask>0,dtype='float32')
        user_vert_mask = np.array(user_vert_mask>0,dtype='float32')
        vert_subvert_mask_input = np.array(vert_subvert_mask_input>0,dtype='float32')
        
        rw_vert = user_vert_num/(user_vert_num.sum(axis=-1).reshape((len(user_vert_num),1))+10**(-8)) #(bz,18)
        rw_subvert = user_subvert_num/(user_subvert_num.sum(axis=-1).reshape((len(user_subvert_num),1))+10**(-8)) #(bz,300)
        
        
        
        rw_vert = rw_vert.reshape((rw_vert.shape[0],1,rw_vert.shape[1]))
        rw_subvert = rw_subvert.reshape((rw_subvert.shape[0],1,rw_subvert.shape[1])) #(bz,1,18)
        
        rw_vert = (rw_vert*vert).sum(axis=-1)
        rw_subvert = (rw_subvert*subvert).sum(axis=-1)
        
        train_mask = np.random.uniform(0,1,size=(ed-start,1)) > self.mask_prob
        train_mask = np.array(train_mask,dtype='float32')
        
        rw_vert = rw_vert*train_mask
        rw_subvert = rw_subvert*train_mask



        return ([title,vert,subvert,user_title, user_vert,user_vert_mask,user_subvert,user_subvert_mask,vert_subvert_mask_input,user_vert_num,user_subvert_num,rw_vert,rw_subvert],[label])
    
    
class get_hir_user_generator(Sequence):
    def __init__(self,news_emb,news_vert,news_subvert,news_entity, clicked_news,batch_size):
        self.news_emb = news_emb
        self.vert = news_vert
        self.subvert = news_subvert
        self.entity = news_entity
        
        self.clicked_news = clicked_news

        self.batch_size = batch_size
        self.ImpNum = self.clicked_news.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))

    
    def __get_news(self,docids):
        news_emb = self.news_emb[docids]
        vert = self.vert[docids]
        subvert = self.subvert[docids]
        entity = self.entity[docids]
        return news_emb, vert, subvert, entity
    
    
    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
            
        clicked_ids = self.clicked_news[start:ed]
        user_title, user_vert, user_subvert, user_entity = self.__get_news(clicked_ids)
        
        vert_subvert_mask_input = np.zeros((len(user_subvert),len(category_dict),len(subcategory_dict),))
        for bid in range(len(user_subvert)):
            for nid in range(len(user_subvert[bid])):
                sv = user_subvert[bid][nid]
                if sv ==0:
                    continue
                sv -= 1
                vert_subvert_mask_input[bid,:,sv] = 1
        vert_subvert_mask_input = vert_subvert_mask_input*vert_subvert_mask_table

        
        
        user_vert = keras.utils.to_categorical(user_vert,len(category_dict)+1)
        user_vert = user_vert.transpose((0,2,1))
        user_vert = user_vert[:,1:,:]
        user_vert_mask = user_vert.sum(axis=-1)
        
        
        user_subvert = keras.utils.to_categorical(user_subvert,len(subcategory_dict)+1)
        user_subvert = user_subvert.transpose((0,2,1))
        user_subvert = user_subvert[:,1:,:]
        user_subvert_mask = user_subvert.sum(axis=-1)
        
        user_vert_num = np.array(user_vert.sum(axis=-1),dtype='int32')
        user_subvert_num = np.array(user_subvert.sum(axis=-1),dtype='int32')
        
        user_subvert_mask = np.array(user_subvert_mask>0,dtype='float32')
        user_vert_mask = np.array(user_vert_mask>0,dtype='float32')
        vert_subvert_mask_input = np.array(vert_subvert_mask_input>0,dtype='float32')

        return [user_title, user_vert,user_vert_mask,user_subvert,user_subvert_mask,vert_subvert_mask_input,user_vert_num,user_subvert_num]

In [21]:
def evaluate_combine2(test_impressions,users,user_subvert_rep,user_vert_rep,user_global_rep,w1,w2,w3):
    AUC = []
    MRR = []
    nDCG5 = []
    nDCG10 =[]
    for i in range(len(test_impressions)):
        labels = test_impressions[i]['labels']
        nids = test_impressions[i]['docs']
        verts = news_vert[nids]
        verts = verts-1
        subverts = news_subvert[nids]
        subverts = subverts-1

        user_gv = user_global_rep[i]
        user_vv = user_vert_rep[i]
        user_svv = user_subvert_rep[i]

        click = users[i]
        
        nv = news_scoring[nids]
        score1 = np.dot(nv,user_gv)
        user_vv = user_vv[verts]
        score2 = (nv*user_vv).sum(axis=-1)
        
        mask2 = []
        for v in verts:
            t = news_vert[click]==(v+1)
            mask2.append(t.sum())
        mask2 = np.array(mask2)
        mask2 = mask2/((click>0).sum()+10**(-6))
        
        
        user_svv = user_svv[subverts]
        score3 = (nv*user_svv).sum(axis=-1)

        mask3 = []
        for svi in range(len(subverts)):
            sv = subverts[svi]
            t = (news_subvert[click]==(sv+1))
            mask3.append(t.sum())
        mask3 = np.array(mask3)
        mask3 = mask3/((click>0).sum()+10**(-6))
        

            
        score1 = np.array(score1)
        score2 = np.array(score2)
        score3 = np.array(score3)

        score = score1*w1+mask2*score2*w2+mask3*score3*w3
        

        auc = roc_auc_score(labels,score)
        mrr = mrr_score(labels,score)
        ndcg5 = ndcg_score(labels,score,k=5)
        ndcg10 = ndcg_score(labels,score,k=10)
    
        AUC.append(auc)
        MRR.append(mrr)
        nDCG5.append(ndcg5)
        nDCG10.append(ndcg10)

    return AUC, MRR, nDCG5, nDCG10

In [22]:
Res = []

In [ ]:
model,news_encoder,user_encoder,rews = create_model(category_dict,subcategory_dict,title_word_embedding_matrix,entity_emb_matrix)
Res.append({'AUC':[],'MRR':[],'nDCG5':[],'nDCG10':[]})
train_generator = get_hir_train_generator(0.9999,news_info,index2nid,news_vert,news_subvert,news_entity_np,news_entity,train_user['click'],train_user_id,train_sess,train_label,16)
model.fit_generator(train_generator,epochs=4,verbose=3)

for i in range(1):
    model.fit_generator(train_generator,epochs=1,verbose=2)
    news_scoring = news_encoder.predict(news_info,verbose=True)
    test_user_generator = get_hir_user_generator(news_scoring,news_vert,news_subvert,news_entity_np,test_user['click'],32)
    
    AUC = []
    MRR = []
    nDCG5 = []
    nDCG10 = []

    for i in range(int(np.ceil(len(test_user['click'])/1000))):
        start = i*1000
        ed = (i+1)*1000
        ed = min(ed,len(test_user['click']))
        test_user_generator = get_hir_user_generator(news_scoring,news_vert,news_subvert,news_entity_np,test_user['click'][start:ed],32)
        user_subvert_rep,user_vert_rep,user_global_rep = user_encoder.predict_generator(test_user_generator,verbose=False)
        a,m,n5,n10 = evaluate_combine2(test_impressions[start:ed],test_user['click'][start:ed],user_subvert_rep,user_vert_rep,user_global_rep,0.15,0.15,0.7)
        AUC += a
        MRR += m
        nDCG5 += n5
        nDCG10 += n10

        print(np.array(AUC).mean(),np.array(MRR).mean(),np.array(nDCG5).mean(),np.array(nDCG10).mean())



    #     break
    AUC = np.array(AUC)
    MRR = np.array(MRR)
    nDCG5 = np.array(nDCG5)
    nDCG10 = np.array(nDCG10)

    AUC = AUC.mean()
    MRR = MRR.mean()
    nDCG5 = nDCG5.mean()
    nDCG10 = nDCG10.mean()
    
    Res[-1]['AUC'].append(AUC)
    Res[-1]['MRR'].append(MRR)
    Res[-1]['nDCG5'].append(nDCG5)
    Res[-1]['nDCG10'].append(nDCG10)









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/4


In [ ]:
Res